## 该Notebook是V13版本的OpenFISH decoding代码，openDecode的版本为V2.0 beta，特性如下：  

* 接受的输入为depth后的czi文件；
* 全程都可以在notebook内部完成 
* 使用的包见 D:/openFISH/Decoding/openDecode
* 主要包含图像预处理(preprocess)-图像对齐(registration)-信号点识别(spot detection)-细胞分割(segmentation)-解码(decoding)-生成表达矩阵(matrixization)五个部分
* 以上五个部分有对应的‘代号’，可以用于跳过运行或强制运行.(具体的跳过或者强制运行命令见下)
| Procedure | Code |
|:---------|:---------:|
| preprocess | 0 |
| registration | 1 |
| spot detection | 2 |
| segmentation | 3 |
| decoding | 4 |
| matrixization | 5 |

___

<div class="alert alert-block alert-info">
<b>提示：</b>拍图时不要设置通道位移参数， 拍图请仔细调整曝光时间和曝光强度，避免过曝，淬灭或过暗。拍图通道选择无要求，但需保证和CodeBook一致</div>

<div class="alert alert-block alert-info">
<b>提示：</b>请至少使用景深扩展处理图像，最好不要进行拼接(到位)，这样会使图像非马赛克化(is_mosiac = False)。但是该解码程序支持非拼图(包括单张小图或拼接好的大图)</div>

<div class="alert alert-block alert-success">
<b><h2> 1.填写参数</b></h2> 
</div>

<div class="alert alert-block alert-info">
<b>提示:</b>CodeBook格式如下(逗号分隔)：</div>
支持全单色编码，全双色编码，单双色混合编码

| gene | RO1 | RO2 | Round |
|---------:|---------:|---------:|---------:|
| Cck | AF488 | AF546 | R1 |
| Rgs5 | AF488 | AF594 | R1 |
| Nfib | Cy5 | Cy5 | R1 |
| Slc17a6 | AF546 | AF546 | R4 |

In [1]:
import sys
sys.path.append("/media/duan/DuanLab_Data/openFISH/Decode")

from openDecode_Black import OpenDecoder, config

para = config.Para(
    
    filename = {
        'R1': '/media/duan/sda2/MALDI/Data/20250928_Gel26Test/20250928_change26pad_test_gel_R1_depth.czi',
        'R2': '/media/duan/sda2/MALDI/Data/20250928_Gel26Test/20250928_change26pad_test_gel_R2_depth.czi',
        'R3': '/media/duan/sda2/MALDI/Data/20250928_Gel26Test/20250928_change26pad_test_gel_R3_depth.czi',
        'R4': '/media/duan/sda2/MALDI/Data/20250928_Gel26Test/20250928_change26pad_test_gel_R4_depth.czi',
        'R5': '/media/duan/sda2/MALDI/Data/20250928_Gel26Test/20250928_change26pad_test_gel_R5_depth.czi',
        'R6': '/media/duan/sda2/MALDI/Data/20250928_Gel26Test/20250928_change26pad_test_gel_R6_depth.czi',
        'R7': '/media/duan/sda2/MALDI/Data/20250928_Gel26Test/20250928_change26pad_test_gel_R7_depth.czi',
        'R8': '/media/duan/sda2/MALDI/Data/20250928_Gel26Test/20250928_change26pad_test_gel_R8_depth.czi',
        'R9': '/media/duan/sda2/MALDI/Data/20250928_Gel26Test/20250928_change26pad_test_gel_R9_depth.czi',
        'R10': '/media/duan/sda2/MALDI/Data/20250928_Gel26Test/20250928_change26pad_test_gel_R10_depth.czi',
        'R11': '/media/duan/sda2/MALDI/Data/20250928_Gel26Test/20250928_change26pad_test_gel_R11_depth.czi',
    },
    
    codebook_path = '/media/duan/sda2/MALDI/Data/20250928_Gel26Test/Test26.csv', # CodeBook路径，需为逗号分隔文件
    output_path = 'Result', # 输出结果路径
    anchor_channel = 'AF405', # 用于对齐的通道名称，默认为AF405。也可以替换为其它的。
    
    extra = None,
    # 如果没有额外的轮，填写 extra = None或extra = {}等即可
    # rRNA一定设置key为rRNA，其它的可以根据实际情况设置，例如P16，GFAP，MRC1...
    # extra = {
    #     'rRNA': {
    #         'filepath': '/media/duan/DuanLab_Data/openFISH/Decode/TestData/20250118_P16_13M_58_V5_8_ABA109_GFP_rRNA_depth_new.czi',
    #         'channel': ['AF405', 'AF546'] # 这里需要列出涉及到的通道名称
    #     },
    #     "P16": {
    #         'filepath': '/media/duan/DuanLab_Data/openFISH/Decode/TestData/20250118_P16_13M_58_V5_8_ABA109_GFP_rRNA_depth.czi',
    #         'channel': ['AF405', 'AF488'] # 这张图像有AF405，AF488，AF546三个通道，但是只需要前两个通道
    #     }
    # },
    
    run_deblur = True, # 是否运行去模糊，所有的extra图像会无视这个参数，都不运行去模糊
    run_BaSiC = True, # 是否运行BaSiC用于去除小图阴影，非拼图无视这个参数，extra接受这个参数
    objective = '20X', # 物镜倍率，影响通道间位移，像素点物理距离参数。可填任意参数，只要在translation_matrix和pixel_scaler提供对应的键即可
    threads = 48, # 使用的线程数，最大不会超过电脑最大线程数-1
    run_basic_clustering = False, # 在运行matrixization后是否进行简单的聚类分析，基因数大于60时无视这个参数，默认运行
    
    # 该程序设定了两个概念，procedure和progress；progress指的是每一轮，每一通道的处理情况，在output_path中的tmp文件夹中会生成一个yaml文件用于储存进度，
    # 程序会自动根据yaml的情况在断点处运行。
    # procedure指的是从图像预处理到生成表达矩阵的五个部分，每一个部分有对应的代号，可以跳过或强制运行
    # 0: preprocess, 1: stitching&regietration, 2: point detection, 3: segmentation, 4: decoding, 5: matrixization
    # 具体的使用例子见最后
    skip_procedure = [3],
    force_procedure = [0,1,2,4,5],
   # 这些参数通常可以不用管，但如果某次实验有一些新的东西引入，可以按需修改
    translation_matrix = {
        "20X":{
            "DAPI": [0.0, 0.0], "AF405": [0.0, 0.0], "AF488": [-0.38+5.38, 0.25], "AF546": [-3.61+5.38,0.12], "AF594": [5.38,0.0], "Cy5": [-2.54+5.38, -0.10], "Cy7": [-2.15+5.38,-0.39], "DIC": [5.38, 0.0]
        },
        "40X":{
            "DAPI": [0.0 ,0.0], "AF405": [0.0, 0.0], "AF488": [-0.44, 0.85], "AF546": [-4.48, -0.03], "AF594": [0.0, 0.0], "Cy5": [-2.68, 0.28], "Cy7": [-1.94, -0.03]
        }
    },
    
    pixel_scaler = {
        '20X': 0.325,
        '40X': 0.1625
    }
    )


OD = OpenDecoder(para)

<div class="alert alert-block alert-success">
<b><h2> 2.程序运行(已列出可以修改的参数)</b></h2> 
</div>

In [2]:
# 0
OD.runPreprocess()
# 1
OD.runRegistration()

# 2
# 参数解释见https://weigertlab.org/spotiflow/api.html#spotiflow.model.spotiflow.Spotiflow.predict
OD.runSpotiflow(
    model_name = "hybiss",
    intensity_threshold = {
        'AF488': 100, 'AF546': 100, 'AF594': 100, 'Cy5': 100, 'Cy7': 100
    },
    prob_thresh = 0.40
)

# 3
OD.runSegmentation(
    # 参数解释见https://github.com/stardist/stardist
    stardist_kwargs = {
                            'prob_thresh': 0.5,
                            'nms_thresh': 0.4,
                            'trained_model': '2D_versatile_fluo',
                            'sigma': 2.5
                        },
    # 参数解释见https://cellpose.readthedocs.io/en/latest/restore.html
    cellpose_kwargs = {
                            'restore_type': 'deblur_cyto3',
                            'diameter': 45.,
                            'flow_threshold': 1.,
                            'cellprob_threshold': -6.,
                        }
)

# 4
# 先找出高置信点训练Ramdom Forest Classifier，然后用训练好的Model去预测所有点
# 参数是优化过的，一般不需要更改
OD.runDecoding(
    # Training part
    dist_threshold = 1.0, # 考虑两个点可能来自同一个RCP的最小距离(pixel)
    prob_threshold = 0.2, # 考虑两个点可能来自同一个RCP的最小概率(Spotiflow的输出)差距
    # Predicting part
    prob_diff_threshold = 0.5,  # 同一个点可能预测出来后属于多个class，top1的class的概率至少比top2的大prob_diff_threshold才保留这个预测结果
    qv_ratio_threshold = 0.85 # 如果一个点被多个组合使用则top1的qv值乘qv_ratio_threshold后需要比top2 qv值大，否则丢弃该点
)

# 5
OD.runMatrixization(
    buffer_radius = 3,  # 细胞核分割的buffer距离，单位为um
    points_distance_threshold = 17.0, # 过滤点的距离参数，只有在没有细胞分割的时候有效，其它时候为细胞直径的平均数
) # 该函数同时也用于生成Xenium Explorer文件

[INFO] (openDecode_Black.main) Force run preporcess procedure
[INFO] (openDecode_Black.preprocess) Preprocessing transcripts cycles
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [05:39<00:00, 30.86s/it]
[INFO] (openDecode_Black.main) Force run registration&stitching procedure
[INFO] (openDecode_Black.registration) Trying MIST with ncc_threshod = 0.05
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:02<00:00, 18.08it/s]
[INFO] (openDecode_Black.registration) Registering tiles...
100%|█████████████████████████████████████████████████████████████████████████████

INFO:spotiflow.model.spotiflow:Loading pretrained model: hybiss


Running Spotiflow: R11_Cy7: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [16:12<00:00,  9.72s/it]
[INFO] (openDecode_Black.main) Skip segmentation procedure
[INFO] (openDecode_Black.main) Force run decoding procedure
[INFO] (openDecode_Black.decoding) Decoding R1...
[INFO] (openDecode_Black.decoding) Preparing training set...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 426182/426182 [00:05<00:00, 79520.82it/s]
[INFO] (openDecode_Black.decoding) Training Random Forest Classifier...
[INFO] (openDecode_Black.decoding) Predicting spots...
[INFO] (openDecode_Black.decoding) Removing multi used spots...
100

INFO     no axes information specified in the object, setting `dims` to: ('c', 'y', 'x')                           


[INFO] (openDecode_Black.matrixization) Resolving 49038 segmentation conflicts
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49038/49038 [00:04<00:00, 9859.74it/s]
[INFO] (openDecode_Black.matrixization) Fitering transcripts within distance  14.5 um...
[INFO] (openDecode_Black.matrixization) Aggregating transcripts into cell_boundaries
[INFO] (sopa.segmentation.aggregation) Aggregating transcripts over 32416 cells


[########################################] | 100% Completed | 2.30 ss


[INFO] (openDecode_Black.matrixization) Running basic clustering...


INFO     The Zarr backing store has been changed from None the new file path: Result/raw_sdata.zarr                


[WARNING] (openDecode_Black.explorer.converter) The cell IDs in the table or shapes are not valid Xenium Explorer IDs. They will be replaced by integers starting from 0 in the output files.
[INFO] (openDecode_Black.explorer.table) Writing table with 57 columns
[INFO] (openDecode_Black.explorer.table) Writing 1 cell/observations categories: region
[INFO] (openDecode_Black.explorer.shapes) Writing 32416 cell polygons
[INFO] (openDecode_Black.explorer.points) Writing 1224631 transcripts
[INFO] (openDecode_Black.explorer.points)    > Level 0: 1224631 transcripts
[INFO] (openDecode_Black.explorer.points)    > Level 1: 306157 transcripts
[INFO] (openDecode_Black.explorer.points)    > Level 2: 76539 transcripts
[INFO] (openDecode_Black.explorer.points)    > Level 3: 19134 transcripts
[INFO] (openDecode_Black.explorer.points)    > Level 4: 4783 transcripts
[INFO] (openDecode_Black.explorer.images) Writing multiscale image with procedure=semi-lazy (load in memory when possible)
[INFO] (openDeco

<div class="alert alert-block alert-warning">
<b><h3>关于跳过步骤或强制执行步骤的example</b></h3>
</div>

| Procedure | Code |
|:---------|:---------:|
| preprocess | 0 |
| registration | 1 |
| spot detection | 2 |
| segmentation | 3 |
| decoding | 4 |
| matrixization | 5 |

___

1.**假设一共11轮图像，今天拍了6轮，想先解码，也就是说你希望运行完解码即可，不用生成最终的表达矩阵，并且你希望下一次添加新的输入后不会重复运行**    
    
    第一次运行填写
     filename = {
        'R1': 'Demo_R1_depth.czi',
        'R2': 'Demo_R2_depth.czi',
        'R3': 'Demo_R3_depth.czi',
        'R4': 'Demo_R4_depth.czi',
        'R5': 'Demo_R5_depth.czi',
        'R6': 'Demo_R6_depth.czi',
    },
    ...
    skip_procedure: = [5], # 这里不跳过maxtrixization也是可以的
    
    第二次运行填写
    filename = {
        'R1': 'Demo_R1_depth.czi',
        'R2': 'Demo_R2_depth.czi',
        'R3': 'Demo_R3_depth.czi',
        'R4': 'Demo_R4_depth.czi',
        'R5': 'Demo_R5_depth.czi',
        'R6': 'Demo_R6_depth.czi',
        'R7': 'Demo_R7_depth.czi',
        'R8': 'Demo_R8_depth.czi',
        'R9': 'Demo_R9_depth.czi',
        'R10': 'Demo_R10_depth.czi',
        'R11': 'Demo_R11_depth.czi',
    },
    ...
    skip_procedure: = []
    
2.**假设运行完全部程序后，你手动运行了某些图的信号点识别代码，你希望重新运行程序，你需要填写**     

    skip_procedure = [0,1,2,3],
    force_procedure = [4,5]
    
    *单独运行Spotiflow代码如下：
    from openDecode.spotdetection import _run_single_spotiflow
    from spotiflow.model import Spotiflow
    import tifffile
    
    img = tifffile.imread("Demo_R1_Cy7.tif")
    
    df = _run_single_spotiflow(img,
                          model = Spotiflow.from_pretrained('hybiss'),
                          intensity_threshold = 100,
                          prob_thresh = 0.40,
                          n_tiles = (5, 5),
                          min_distance = 1, 
                          exclude_border = True,
                          scale = None,
                          subpix = True,
                          peak_mode ='fast',
                          normalizer = 'auto',
                          verbose = True,
                          device = 'cuda')
                          
    df.to_parquet(Demo_R1_Cy7.parquet)
    
3.**假设运行完全部程序后，你对片子进行了额外的抗体染色(GFAP)，你现在想添加进去，你需要填写**     

    extra = {
        'rRNA': {
            'filepath': 'Demo_GFP_rRNA_depth_new.czi',
            'channel': ['AF405', 'AF546'] 
        },
        "GFAP": {
            'filepath': 'Demo_GFAP_depth.czi',
            'channel': ['AF405', 'Cy5'] 
        }
    },
    
另外填写

    skip_procedure = [2,3,4], # 不跳过5，因为根据progress.yaml文件，步骤5中的繁琐步骤会直接跳过，只用于生成带GFAP的Xenium Explorer文件
    force_procedure = []
    
4.**假设运行完全部程序后，你想修改里面的一些参数后重新运行，可以按照以下的基本逻辑来决定**  

    decoding依赖spot detection，所以重新运行spot detection后也一定重新运行decoding  
    matrixization依赖decoding，所以重新运行decoding后也一定重新运行matrixization  

        skip_procedure = [0,1,3],
        force_procedure = [2,4,5]

    matrixization同样也依赖segmentation，所以重新运行segmentation一定重新运行matrixization

        skip_procedure = [0,1,2],
        force_procedure = [3,5]

    除filename和extra外，其它所有修改参数后运行都需要添加到force_procedure中，因为程序不会检测参数修改

<div class="alert alert-block alert-warning">
<b><h3>一些补充说明</b></h3>
</div>

1. 如果最后生成的图片含有三个通道（例如DAPI, rRNA, GFP），Xenium Explorer默认会无视ome.tiff中的通道名称，在软件中显示R，G，B。但是可以通过在软件里再次add image的方法来方便图像查看，此时的图片会显示每个通道对应的名称。通道名称为你输入的extra的key + _ + 通道。例如 rRNA_AF546或GFAP_Cy5

2. 结果文件含有Registration, Segmentation, tmp三个文件夹，和一个spatialdata(raw_sdata.zarr)文件，一个Xenium Explorer文件和一个Decoded_transcripts.parquet（储存着解码后的点的信息，其中有一列是qv值，代表着点的置信度，qv < 20意味着该点根据点与点之间的距离被标记了），和一个adata.h5ad文件。Registration中含有两个子文件夹分别用于储存拼接前和拼接后的图片，拼接前的图片以npy格式保存，拼接后图片以tiff格式保存，Spotiflow结果以parquet格式保存。另外根据输入还会生成1-3张morphology结果的tiff图片。Segmentation中含有细胞分割的结果。tmp文件夹中包含很多中间文件，方便断点开始或数据检查。

3. 原始图像会进行简单的clip以去除过曝的点，然后会使用richardson_lucy进行deblur，psf是估算出来的Gaussian psf。然后正常运行BaSiC。
    
4. 图像拼接和对齐的方法没有改变，只是对齐的参数进行了一些调整，兼顾了速度和准确度。

5. 信号点识别目前只保留的Spotiflow，**如果有需要**，我可以把RS-FISH也添加进来。

6. 信号点的解码目前有两步，第一步选出置信度特别高的点用于训练随机森林模型，然后用这个模型去预测所有的点。如果只含有单色编码，那么就会直接将点转化成对应基因，把Spotiflow的最后结果prob * 40作为qv。

7. 最后会生成表达矩阵并进行简单聚类(如果基因数大于60或者run_basic_clustering = True). rRNA和DAPI的分割依旧是按照rRNA为主，DAPI为辅的原则merge到一起。

8. **再次强调，程序能够检测输入文件的更改，无法检测其它输入的更改，如果有函数参数发生改变等，请务必调整skip_procedure和force_procedure这两个参数。**

9. 设置了不合理的skip_procedure和force_procedure程序会提示警告，所以仔细查看输出的字段。

10. rRNA只支持两个通道输入，不要设置第三个通道，如果rRNA的图片含有其余通道的信息，可以将其设置为

        extra = {
            'rRNA': {
                'filepath': '/media/duan/DuanLab_Data/openFISH/Decode/TestData/20250118_P16_13M_58_V5_8_ABA109_GFP_rRNA_depth_new.czi',
                'channel': ['AF405', 'AF546'] # 这里需要列出涉及到的通道名称
            },
            "Other": {
                'filepath': '/media/duan/DuanLab_Data/openFISH/Decode/TestData/20250118_P16_13M_58_V5_8_ABA109_GFP_rRNA_depth_new.czi',
                'channel': ['AF405', 'AF488'] # 同样的图片，但是这次只需要AF405和AF488
                

10. *还有其它问题就问 xinyoung_li@genetics.ac.cn*

<div class="alert alert-block alert-warning">
<b><h3>另外，如果需要运行多个notebook，为避免运行完占用内存/显存不释放，可以运行下面的代码</b></h3>
</div>

In [ ]:
%reset -f
import gc
gc.collect()

try:
    import cupy as cp
    cp.get_default_memory_pool().free_all_blocks()
    cp.get_default_pinned_memory_pool().free_all_blocks()

except:
    pass

try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        
except:
    pass